In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_lab_df,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
)
from web import for_website

alt.data_transformers.disable_max_rows() # Allow using rows more than 5000

In [ ]:
df = read_full_lab_df()
df

In [ ]:
df

In [ ]:
loinc_df = read_loinc_df().set_index('LOINC')

LOINC_NAMES = loinc_df["Name"].unique().tolist()
LOINC_IDS = loinc_df.index.values.tolist()
LOINC_UNITS = loinc_df['Units'].values.tolist()
LOINC_MAP = { LOINC_IDS[i]: f'{LOINC_IDS[i]}, {LOINC_NAMES[i]} ({LOINC_UNITS[i]})' for i in range(len(LOINC_IDS)) }

# print(list(LOINC_MAP.values()))

loinc_df

In [ ]:
def process_labs_df(df_lb):
    
    # Negative values to zeros
    df_lb.loc[df_lb['mean_value_all'] < 0, 'mean_value_all'] = np.nan
    df_lb.loc[df_lb['stdev_value_all'] < 0, 'stdev_value_all'] = np.nan
    df_lb.loc[df_lb['mean_value_ever_severe'] < 0, 'mean_value_ever_severe'] = np.nan
    df_lb.loc[df_lb['stdev_value_ever_severe'] < 0, 'stdev_value_ever_severe'] = np.nan
    df_lb.loc[df_lb['mean_log_value_ever_severe'] < 0, 'mean_log_value_ever_severe'] = np.nan
    df_lb.loc[df_lb['stdev_log_value_ever_severe'] < 0, 'stdev_log_value_ever_severe'] = np.nan
    
    # Remove white space which shouldn't be provided
    df_lb['loinc'] = df_lb['loinc'].apply(lambda loinc: loinc.replace(' ', ''))
    
    # Add readable names for LOINC and units
    df_lb['loinc_name'] = df_lb['loinc'].apply(lambda loinc:
        LOINC_MAP[loinc]                                     
        # loinc_df.at[loinc, "Name"] + " (" + loinc_df.at[loinc, "Units"] + ")"
    )
    
    # Upper and under bound for values
    df_lb["upper_all"] = df_lb['mean_value_all'] + df_lb['stdev_value_all'] 
    df_lb["under_all"] = df_lb['mean_value_all'] - df_lb['stdev_value_all']
    df_lb["upper_ever_severe"] = df_lb['mean_value_ever_severe'] + df_lb['stdev_value_ever_severe'] 
    df_lb["under_ever_severe"] = df_lb['mean_value_ever_severe'] - df_lb['stdev_value_ever_severe']
    
    # Sort by country
    df_lb = df_lb.sort_values(by=['country'])
    
    # TODO: Data filter for better performance
    
    return df_lb

In [ ]:
pdf = process_labs_df(df)
pdf.head()

In [ ]:
COUNTRY_NAMES = list(get_country_color_map().keys())
COUNTRY_COLORS = list(get_country_color_map().values())
COUNTRY_COLOR_MAP = {COUNTRY_NAMES[i]: COUNTRY_COLORS[i] for i in range(len(COUNTRY_NAMES))}

COUNTRY_COLOR_MAP

In [ ]:
SITE_NAMES = list(get_siteid_color_maps().keys())
SITE_COLORS = list(get_siteid_color_maps().values())
SITE_COLOR_MAP = {SITE_NAMES[i]: SITE_COLORS[i] for i in range(len(SITE_NAMES))}

SITE_COLOR_MAP

In [ ]:
def lab_plot(
    df=None,
    loinc=list(LOINC_MAP.values())[0], # ALT
    severity='all', # either 'all' or 'ever_severe'
    width=700, 
    height=400,
    bar_size=14,
    point_size=30,
    no_axis_title=False,
    no_legend=False, 
    legend_columns=None,
    y_domain_top=None, 
    y_domain_bottom=None
):
    if(df is None):
        print('No Data Frame Suggested.')
        return
    
    # data field and title definition
    loinc_code = 'loinc'
    loinc_name = 'loinc_name'
    color_field = 'siteid'
    x_field = 'days_since_admission'
    mean_field = f'mean_value_{severity}'
    num_pat_field = f'num_patients_{severity}'
    
    x_field_title = 'Days since positive'
    mean_field_title = loinc #'Mean value'
    
    site_color = alt.Color(
        f"{'siteid'}:N", 
        title='Site ID',
        scale=alt.Scale(domain=SITE_NAMES, range=SITE_COLORS)
    )
        
    country_color = alt.Color(
        f"{'country'}:N", 
        title=None,
        scale=alt.Scale(domain=COUNTRY_NAMES, range=COUNTRY_COLORS),
        legend=None
    )

    # data selection
    base = (
        alt.Chart(df)
            # TODO: Move this part to data preprocessing for the better performance
            .transform_filter(alt.datum[x_field] >= 0)
            .transform_filter(alt.datum[x_field] <= 30)
            .transform_filter(alt.datum[num_pat_field] >= 10)
    )
    
    legend_selection = alt.selection_multi(fields=['siteid'], bind="legend")
    
    date_brush = alt.selection(type="interval", encodings=['x'])
    
    input_dropdown = alt.binding_select(options=list(LOINC_MAP.values()))
    selection = alt.selection_single(
        fields=['loinc_name'], 
        bind=input_dropdown, 
        name='Lab ', 
        init={'loinc_name': loinc}
    )
    
    y_zoom_top = alt.selection(type="interval", bind='scales', encodings=['y'])
    y_zoom_bottom = alt.selection(type="interval", bind='scales', encodings=['y'])
    
    nearest = alt.selection(type="single", nearest=True, on="mouseover", fields=[x_field], empty='none', clear="mouseout", name="nearest_selector")
    nearest_rule = alt.Chart(df).mark_rule(color="red").encode(
        x=f"{x_field}:Q",
        size=alt.value(0.5)
    ).transform_filter(
        nearest
    )
    
    tooltip = [
        alt.Tooltip('siteid', title="Site ID"),
        alt.Tooltip('country', title="Country"),
        alt.Tooltip(mean_field, title="Lab value", format=".2f"),
        alt.Tooltip(num_pat_field, title="Number of patients"),
        alt.Tooltip(x_field, title="Days since positive")
    ]
    
    """
    Lab mean values in line/dot plot
    """
    y_title = None if no_axis_title else mean_field_title
    no_x_axis = False

    x_axis = (
        alt.Axis(grid=True, labels=False, ticks=False, domain=True)
            if no_x_axis 
            else alt.Axis(grid=True, labels=True, ticks=True, domain=True, tickMinStep=1)
    )
    y_scale = (
        alt.Scale(zero=False, domain=y_domain_bottom, type="linear") 
            if y_domain_bottom != None 
            else alt.Scale(zero=False, type="linear")
    )

    circle = (
        base
            .mark_circle(size=point_size, opacity=0.7)
            .encode(
                x=alt.X(
                    f"{x_field}:Q",
                    title=None,
                    scale=alt.Scale(zero=False, nice=False, padding=10),
                    axis=x_axis,
                ),
                y=alt.Y(
                    f"{mean_field}:Q",
                    title=y_title,
                    scale=y_scale,
                    axis=alt.Axis(format='r')
                ),
                color=site_color,
                tooltip=tooltip
            )
    )
    
    line = circle.mark_line(size=2, opacity=0.5)

    errorbar = (
        circle
            .mark_errorbar()
            .encode(
                y=alt.Y("mean(upper_ever_severe):Q", title=""), 
                y2="mean(under_ever_severe):Q",
                size=alt.value(1),
                opacity=alt.value(1)
            )
    )
    
    lab_value_plot = (
        alt.layer(circle, line, nearest_rule)
            .properties(height=height, width=width)
            .add_selection(y_zoom_top)
            .transform_filter(date_brush)
    )


    """
    # of patients in bar charts
    """
    num_pat_field_title = None if no_axis_title else "# of patients"

    bar = base.mark_bar(size=bar_size).encode(
        y=alt.Y(
            f"sum({num_pat_field}):Q", 
            title=num_pat_field_title,
            axis=alt.Axis(
                format='r',
                tickMinStep=1
            )
        ),
        x=alt.X(
            f"{x_field}:Q",
            title=x_field_title,
            scale=alt.Scale(zero=False, nice=False, padding=10),
            axis=alt.Axis(
                grid=True,
                labels=True,
                ticks=True,
                domain=True
            )
        ),
        color=site_color,
        order=alt.Order(
            'country',
            sort='ascending'
        ),
        tooltip=tooltip
    )
    
    middle_chart = (
        (bar + nearest_rule)
            .properties(height=130, width=width)
            .add_selection(date_brush)
    )
    
    """
    # of sites in bar charts
    """
    bar_bottom_y_title ="# of sites"

    bottom_bar = base.mark_bar(size=bar_size).encode(
        x=alt.X(
            f"{x_field}:Q",
            title=x_field_title,
            scale=alt.Scale(
                zero=False,
                nice=False,
                padding=10
            )
        ),
        y=alt.Y(
            f"sum({num_pat_field}):Q", 
            title=bar_bottom_y_title,
            axis=alt.Axis(
                format='r', 
                tickMinStep=1
            )
        ),
        color=country_color
    )

    bottom_chart = bottom_bar.properties(
        height=130,
        width=width
    )

    result_vis = (
        alt.vconcat(
            lab_value_plot, 
            middle_chart,
            spacing=5
        ).properties(
            title={
                "text": severity.replace('_', ' ').title(),
                'subtitle': get_visualization_subtitle(data_release='2020-06-16'),
                'subtitleColor': 'gray'
            }
        ).resolve_scale(
            y="independent", 
            x="independent",
            color="shared"
        )
        .add_selection(selection)
        .add_selection(nearest)
        .transform_filter(selection)
        .add_selection(legend_selection)
        .transform_filter(legend_selection)
    )
    
    return result_vis

In [ ]:
severity_all = lab_plot(df=pdf, severity='all', width=300)
severity_ever_severe = lab_plot(df=pdf, severity='ever_severe', width=300)

# display(severity_all)

final = alt.hconcat(severity_all, severity_ever_severe, spacing=15).resolve_scale(y="shared")

final = apply_theme(final)

final